In [3]:
import sys
sys.path.append("/Users/haleighwright/Desktop/NCEAS/pyriv")
sys.path.append("/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")

In [7]:
import networkx as nx
import numpy as np
import sys
import shapely

from pyriv import graph_prep as GraphBuilder
from pyriv import river_graph as RiverGraph

In [17]:
def deadstarts(self):
    degdict = self.in_degree(self.nodes())
    deadstarts = {}
    for key, value in degdict.iteritems():
        if value==0:
            deadstarts[key] = value
    return deadstarts


In [18]:
def deadends(self):
    """
    For a directed graph, find the nodes that have in edges but no out edges.
    """
    thedict = {}
    # get a dictionary of { node: out_degree }
    degdict = self.out_degree(self.nodes())
    # convert to a 2 column array of node and out_degree
    degarr = np.array(degdict.items())
    # get a boolean index of rows where out_degree==0
    dead_row_ind = (degarr[:,1]==0)
    
    for i in range(0, len(degdict)):
        if dead_row_ind[i] == True:
            thedict[i] = tuple(degarr[:,0][i])
            
    # use that index to get the nodes
    #dead_nodes = degarr[:,0][dead_row_ind]
    return thedict

In [9]:
kusko_g = GraphBuilder.GraphBuilder('../../data/sasap/kusko_region_flowlines.gpickle')

In [19]:
ends_dict = deadends(kusko_g.graph)
starts_dict = deadstarts(kusko_g.graph)

In [23]:
print len(ends_dict)
print len(starts_dict)
print len(kusko_g.graph.nodes())


5533
63807
277425


In [26]:
def add_missing_edges(graph, deadends, deadstarts, threshold):
    for key1, node1 in deadends.iteritems():
        for key2, node2 in deadstarts.iteritems():
            if key1 != key2 and np.linalg.norm(np.subtract(node1,node2)) < threshold:
                graph.add_edge(key1, key2)
                print("added edge from "+str(key1)+" : "+str(node1)+" to "+ str(key2)+" : "+str(node2))

In [27]:
kusko_gs = kusko_g.graph.copy()

In [28]:
add_missing_edges(kusko_gs, ends_dict, starts_dict, 5.0)